# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
csv_path = "../WeatherPy/weather_df.csv"
weather_df = pd.read_csv(csv_path, encoding="utf-8")
weather_df


,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temperature,Wind Speed (mph)
0,Portland,90,US,1579838530,87,45.52,-122.68,54.00,11.41
1,Bredasdorp,81,ZA,1579838709,88,-34.53,20.04,66.00,4.70
2,Mataura,100,NZ,1579838711,42,-46.19,168.86,83.55,3.49
3,Isangel,75,VU,1579838712,58,-19.55,169.27,87.80,17.22
4,Fort Nelson,90,CA,1579838169,92,58.81,-122.70,14.00,3.36
...,...,...,...,...,...,...,...,...,...
570,Hanna,93,CA,1579839664,91,51.63,-111.90,19.99,4.00
571,Burriana,90,ES,1579839666,100,39.89,-0.09,50.00,2.24
572,Champerico,9,GT,1579839368,82,14.30,-91.92,74.01,5.10
573,San Andres,40,CO,1579839669,69,12.58,-81.70,78.80,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)


In [18]:
locations = weather_df[["Latitude", "Longitude"]].astype(float)

humidity = weather_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map